# Convert to TFLite

References
* TensorFlow Lite (TFLite) [developer guide](https://www.tensorflow.org/mobile/tflite/devguide)
* TFLite Optimizing Converter (TOCO) [Python API](https://www.tensorflow.org/api_docs/python/tf/contrib/lite/toco_convert)
* TOCO [Python API documentation](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/lite/toco/g3doc/python_api.md)
* TOCO [command-line examples](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/lite/toco/g3doc/cmdline_examples.md)
* Converting a SavedModel to TFLite [howto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/saved_model/README.md)

## Preliminaries

In [ ]:
project_id = datalab_project_id()
bucket = 'gs://%s-image-classifier-lite' % project_id
!gsutil mb $bucket

## Load the trained graph

In [ ]:
import tensorflow as tf
from tensorflow.python.platform import gfile


def load_graph(graph_filename):
  with tf.gfile.GFile(graph_filename, "rb") as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
  with tf.Graph().as_default() as graph:
    tf.import_graph_def(
      graph_def,
      input_map=None,
      return_elements=None,
      name="prefix",
      op_dict=None,
      producer_op_list=None)
  input_tensor = graph.get_tensor_by_name('prefix/input:0') #input tensor
  output_tensor = graph.get_tensor_by_name('prefix/final_result:0') #output tensor
  return graph, input_tensor, output_tensor

## Exercise: Convert the graph to TFLite format

In [ ]:
def convert_graph(frozen_graph,
                  input_tensor,
                  output_tensor,
                  lite_file):
  with tf.Session(graph=frozen_graph) as sess:
    # Following the TOCO Python API documentation (see below)
    # convert the trained model to TFLite format and save it 
    # into the 'lite_file' file.
    # Documentation: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/lite/toco/g3doc/python_api.md 
    # <YOUR CODE HERE>
    raise NotImplementedError()

## Solution
Display the content of the box below to see the solution to the exercise.

In [ ]:
def convert_graph(frozen_graph,
                  input_tensor,
                  output_tensor,
                  lite_file):
  with tf.Session(graph=graph) as sess:
    tflite_model = tf.contrib.lite.toco_convert(sess.graph_def,
                                                [input_tensor],
                                                [output_tensor])
    with gfile.GFile(lite_file, "wb") as timeline_file:
      timeline_file.write(tflite_model)

## Run the conversion

In [ ]:
tf.logging.set_verbosity(tf.logging.DEBUG)

model_file = 'gs://candies-ml/model/v2/retrained_graph.pb'
graph, input_tensor, output_tensor = load_graph(model_file)

lite_file = '%s/retrained_graph.lite' % bucket
convert_graph(graph, input_tensor, output_tensor, lite_file)

labels_file = 'gs://candies-ml/model/v2/retrained_labels.txt'
!gsutil cp $labels_file $bucket